# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas.core.common import flatten

In [3]:
url = 'https://habr.com'

In [4]:
def page_url(page):
    """Возвращает url страницы с номером page"""
    return url + f'/ru/search/page{page}/'

In [5]:
def get_habr_page_link(query, page):
    """Возвращает ссылки со страницы по заданному запросу"""
    res = requests.get(
        page_url(page),
        params={'q': query}
    )
    if res.status_code != 200:
        return {}
    links = BeautifulSoup(res.text).find_all('a', class_='tm-title__link')
    return {link['href'] for link in links}

In [6]:
def get_query_links(query, pages):
    """Возвращает все ссылки с pages страниц по запросу query"""
    return set(flatten(
        [get_habr_page_link(query, page) for page in range(1, pages+1)]
        ))

In [7]:
def parse_page(link):
    """Парсит статью по конкретной ссылке"""
    page_url = url + link
    res = requests.get(page_url)
    if res.status_code != 200:
        return {
            'link': page_url,
            'title': '!!! ОШИБКА !!!',
            'text': 'Ошибка доступа к странице'
        }
    try:
        page = BeautifulSoup(res.text)
        title = page.find('h1', class_='tm-title tm-title_h1').span.text
        date = page.find('span', class_='tm-article-datetime-published').time['title']
        text = page.find('div', class_='article-formatted-body').text
        like = int(page.find('span', class_='tm-votes-meter__value').text)
    except Exception as err:
        return {
            'link': page_url,
            'title': '!!! ОШИБКА !!!',
            'text': 'Ошибка парсинга страницы\n' + str(err)
        }
    return {
        'link': page_url,
        'text': text,
        'title': title,
        'date': date,
        'like': like
    }
    

In [8]:
def get_pages_links(queries, pages):
    """Получает все ссылки по запросам queries глубиной pages страниц"""
    links = set()
    links.update(
        *[get_query_links(query, pages) for query in queries]
    )
    return links
    

In [9]:
def get_habr_pages(queries, pages):
    """Формирует необходимый DataFrame"""
    links = get_pages_links(queries, pages)
    df = pd.DataFrame()
    for link in links:
        df = pd.concat(
            [df, pd.DataFrame([parse_page(link)])]
            )
    return df.reset_index(drop=True)

In [10]:
habr_df = get_habr_pages(['python', 'clojure'], 2)
habr_df

,link,text,title,date,like
0,https://habr.com/ru/articles/196418/,Мы сделали это.\r\nРоссийский проект CodeNotes...,CodeNotes — победитель Clojure Cup 2013,"2013-10-04, 21:16",68.0
1,https://habr.com/ru/articles/774918/,"Привет, Хабр! Аргентум на связи, сегодня мы бу...","Змеиная наука: Химия в Python, часть 1","2023-11-19, 14:05",8.0
2,https://habr.com/ru/articles/775440/,В наше время автоматизация стала ключевым факт...,Внедрение Автоматизации в Проект с Python: Шаг...,"2023-11-21, 17:13",2.0
3,https://habr.com/ru/news/766124/,JetBrain совместно с Python Software Foundatio...,JetBrains и Python Software Foundation рассказ...,"2023-10-09, 08:39",8.0
4,https://habr.com/ru/articles/549362/,Всем привет!\n\r\nЯ сделал перевод и набил суб...,Effective Programs (10 Years of Clojure) — Ric...,"2021-03-28, 15:19",4.0
...,...,...,...,...,...
70,https://habr.com/ru/articles/339628/,"\nОбъединяем Websockets, Lisp и функциональное...","Объединяем Websockets, Lisp и функциональное п...","2017-10-08, 23:18",7.0
71,https://habr.com/ru/articles/137608/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
72,https://habr.com/ru/news/775270/,Исследователь из Массачусетского технологическ...,"Исследование: ChatGPT работает лучше на Julia,...","2023-11-21, 10:27",0.0
73,https://habr.com/ru/articles/247889/,В последнее время определенную известность по...,"Clojure — трансдьюсеры, редьюсеры и прочая муть","2015-01-19, 07:28",37.0


In [11]:
habr_df[habr_df['title'].str.contains('ОШИБКА')]

,link,text,title,date,like
17,https://habr.com/ru/articles/104185/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
27,https://habr.com/ru/articles/102197/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
48,https://habr.com/ru/companies/epam_systems/new...,Ошибка парсинга страницы\n'NoneType' object ha...,!!! ОШИБКА !!!,NaN,NaN
51,https://habr.com/ru/companies/raiffeisenbank/n...,Ошибка парсинга страницы\n'NoneType' object ha...,!!! ОШИБКА !!!,NaN,NaN
71,https://habr.com/ru/articles/137608/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
